MediaPipe Tracking Test

In [12]:
import cv2
import mediapipe as mp
import time
import psutil

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
)

# Open webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Error: Cannot access the webcam.")
    exit()

# Benchmarking setup
start_time = time.time()
frame_count = 0
total_pose_time = 0.0
cpu_usages = []

print("📸 Running for 10 seconds...")

while True:
    current_time = time.time()
    if current_time - start_time > 10:
        break

    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to read frame.")
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Measure CPU usage before processing
    cpu_usage = psutil.cpu_percent(interval=None)
    cpu_usages.append(cpu_usage)

    # Pose detection
    t0 = time.time()
    results = pose.process(rgb_frame)
    t1 = time.time()
    pose_time = t1 - t0
    total_pose_time += pose_time
    frame_count += 1

    # Draw landmarks
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2),
        )

    # Overlay metrics
    elapsed = current_time - start_time
    fps = frame_count / elapsed if elapsed > 0 else 0
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    cv2.putText(frame, f"Pose Time: {pose_time * 1000:.1f} ms", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
    cv2.putText(frame, f"CPU: {cpu_usage:.1f}%", (10, 90),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    # Show frame
    cv2.imshow("MediaPipe Pose Benchmark", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

# Summary
total_time = time.time() - start_time
avg_fps = frame_count / total_time if total_time > 0 else 0
avg_pose_time = (total_pose_time / frame_count) * 1000 if frame_count > 0 else 0
avg_cpu = sum(cpu_usages) / len(cpu_usages) if cpu_usages else 0

print("\n=== Benchmark Results (10s) ===")
print(f"Total frames: {frame_count}")
print(f"Total time: {total_time:.2f} s")
print(f"Average FPS: {avg_fps:.2f}")
print(f"Average Pose Detection Time: {avg_pose_time:.2f} ms")
print(f"Average CPU Usage: {avg_cpu:.2f}%")


I0000 00:00:1744814444.672612    4613 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1744814444.673860  134328 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.9-0ubuntu0.2), renderer: Mesa Intel(R) UHD Graphics 630 (CFL GT2)
W0000 00:00:1744814444.737711  134318 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744814444.772894  134315 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


📸 Running for 10 seconds...

=== Benchmark Results (10s) ===
Total frames: 282
Total time: 10.16 s
Average FPS: 27.77
Average Pose Detection Time: 22.41 ms
Average CPU Usage: 9.73%


Yolo Tracking Test

In [13]:
import cv2
import time
import psutil
from ultralytics import YOLO

# Load YOLOv8 model (use 'yolov8n.pt' for speed or 'yolov8s.pt' for better accuracy)
model = YOLO("yolov8n.pt")  # Make sure this model is downloaded or in your dir

# Start webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Cannot access webcam.")
    exit()

# Benchmarking setup
start_time = time.time()
frame_count = 0
total_inference_time = 0.0
cpu_usages = []

print("📸 Running YOLOv8 for 10 seconds...")

while True:
    current_time = time.time()
    if current_time - start_time > 10:
        break

    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to read frame.")
        break

    cpu_usage = psutil.cpu_percent(interval=None)
    cpu_usages.append(cpu_usage)

    # Inference timing
    t0 = time.time()
    results = model(frame, verbose=False)[0]  # No printout
    t1 = time.time()
    inference_time = t1 - t0
    total_inference_time += inference_time
    frame_count += 1

    # Draw detections (only persons)
    for box in results.boxes:
        cls = int(box.cls[0])
        if model.names[cls] == "person":
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, "Person", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Show performance info
    elapsed = current_time - start_time
    fps = frame_count / elapsed if elapsed > 0 else 0
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    cv2.putText(frame, f"Infer Time: {inference_time * 1000:.1f} ms", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
    cv2.putText(frame, f"CPU: {cpu_usage:.1f}%", (10, 90),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    # Display
    cv2.imshow("YOLOv8 Person Detection", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC to break early
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

# Final metrics
total_time = time.time() - start_time
avg_fps = frame_count / total_time if total_time > 0 else 0
avg_infer = (total_inference_time / frame_count) * 1000 if frame_count > 0 else 0
avg_cpu = sum(cpu_usages) / len(cpu_usages) if cpu_usages else 0

print("\n=== YOLOv8 Benchmark (10s) ===")
print(f"Frames Processed: {frame_count}")
print(f"Total Time: {total_time:.2f} s")
print(f"Average FPS: {avg_fps:.2f}")
print(f"Avg Inference Time: {avg_infer:.2f} ms")
print(f"Avg CPU Usage: {avg_cpu:.2f}%")


📸 Running YOLOv8 for 10 seconds...

=== YOLOv8 Benchmark (10s) ===
Frames Processed: 282
Total Time: 10.15 s
Average FPS: 27.79
Avg Inference Time: 12.24 ms
Avg CPU Usage: 8.03%
